In [1]:
import pandas as pd
import datetime

%matplotlib inline
userLog = pd.read_csv("./clean_data/userlog_feature.csv")

In [2]:
#userLog = pd.read_csv("./clean_data/userlog_feature.csv")
userLog['datetime'] = pd.to_datetime(userLog['datetime'])
isTrain = userLog['datetime']<datetime.datetime(2018,2,18)
userLog = userLog[isTrain]

In [3]:
userLog['projects_per_hour'] = userLog['user_clicked_by_hour']/userLog['user_clicked_projects_by_hour']
userLog = userLog[userLog['projects_per_hour']<100]
print(userLog.shape)

(1100701, 13)


In [ ]:
# userLog['datetime'] = userLog.apply(lambda row : datetime.datetime(row['year'], row['month'], row['day'], row['hour']), axis=1)
# userLog['date'] = userLog['datetime'].map(lambda x : x.date())
# userLog['yearmonth'] = userLog['date'].map(lambda x: str(x.year) +'-'+ str(x.month).zfill(2))
# userLog.head(3)

In [4]:
# # Filter out user that is a bot

project_count = userLog.groupby(['userCode']).size()
select_project = set(project_count[project_count < 1000].index)

userLog2 = userLog[userLog['userCode'].isin(select_project)]
print(userLog2.shape)
userLog2.head()

(1066711, 13)


,userCode,project_id,requestedDevice,userAgent,pageReferrer,year,month,day,hour,datetime,user_clicked_by_hour,user_clicked_projects_by_hour,projects_per_hour
0,7717bdc2-ea3e-e8ad-5d6b-178bd71c38b2,7956,Mobile,Android,HomeWebsite,2018,1,1,0,2018-01-01,3,3,1.0
1,7717bdc2-ea3e-e8ad-5d6b-178bd71c38b2,7956,Mobile,Android,HomeWebsite,2018,1,1,0,2018-01-01,3,3,1.0
2,cb5b4b68-cc01-6db6-f54b-4a0f881301c5,5067,Mobile,iPhone,HomeWebsite,2018,1,1,0,2018-01-01,1,1,1.0
3,5f74cef2-0d1e-b619-3564-0955a14e0985,6654,Mobile,iPhone,Google,2018,1,1,0,2018-01-01,1,1,1.0
4,dba8f279-844e-eef6-73ac-22bd7d1353cc,6474,Mobile,iPad,Google,2018,1,1,0,2018-01-01,2,2,1.0


In [ ]:
# min_interacted = 30
# project_count = userLog.groupby(['project_id']).size()
# ignore_project = set(project_count[project_count > min_interacted].index)
# print(len(ignore_project))

# df_filter = userLog[~userLog['project_id'].isin(ignore_project)]
# df_filter.head(3)

In [5]:
userLog2['interacted'] = 1

/home/users/u5722792744_1/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [ ]:
%%time
# Convert the dataframe into a matrix using pivot() and set index of df_train for fast search 
df_train_indexed = userLog2[['userCode', 'project_id', 'interacted']].drop_duplicates()
df_train_pivot = (df_train_indexed.pivot(index = 'userCode', columns = 'project_id', values = 'interacted')
                                  .fillna(0))
df_train_pivot.head(3) 

In [ ]:
import numpy as np
R = df_train_pivot.as_matrix()
user_ratings_mean = np.mean(R, axis = 1)
R_demeaned = R - user_ratings_mean.reshape(-1, 1)


In [ ]:
R.shape
user_ratings_mean.reshape(-1, 1).shape

In [ ]:
%%time
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(R_demeaned, k = 300)

In [ ]:
# import numpy as np
# import gzip
# np.save(open("./var/U_k250.np", "wb"), U)
# np.save(open("./var/sigma_k250.np", "wb"), sigma)
# np.save(open("./var/Vt_k250.np", "wb"), Vt)

In [28]:
sigma = np.diag(sigma)

In [ ]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)
preds_df = pd.DataFrame(all_user_predicted_ratings, columns = df_train_pivot.columns)

In [ ]:
#preds_df = pd.DataFrame(preds_df, columns = df_train_pivot.columns)
preds_df['userCode'] = df_train_pivot.index
preds_df2 = preds_df.reset_index()
#del preds_df['project_id']

### Predicting users

In [ ]:
# New recommend function 
user_seen_projs = userLog.groupby("userCode").agg({"project_id":lambda x:list(x)})
user_seen_projs.loc["00005aba-5ebc-0821-f5a9-bacca40be125"][0]

In [ ]:
projs_id = preds_df.columns
def recommend_project(userId, preds_df=preds_df):
    
    # If not found, return default
    score = preds_df.query("userCode=='%s'"%userId)
    if score.shape[0]==0: return '5858 3738 4201 5452 5678 5764 5067'
    
    # Find out users old projects
    if userId not in user_seen_projs: seen_projs = []
    else: seen_projs = user_seen_projs.loc[userId][0]        
    
    cal_df = pd.DataFrame({"proj_id":projs_id,"score":score.values[0]})
    recommending = cal_df[cal_df['proj_id']!="userCode"].sort_values("score",ascending=False).head(7+len(seen_projs))

    ans = [a for a in recommending['proj_id'] if a not in seen_projs]
    return " ".join([str(i) for i in ans[:7]])
    
recommend_project("00005aba-5ebc-0821-f5a9-bacca40be125")

In [ ]:
# %%time
def predictFromFile(file,submission_name,sep=','):    
    users = pd.read_csv(file,sep=sep)
    print(users.head())
    users['project_id'] = users['userCode'].apply(recommend_project)
    users.to_csv("./submission/%s.csv"%submission_name,index=False)
    
# predictFromFile("./clean_data/test_before_021718_7000row.csv","matrix_factorise_v9_k300_ontest7000")

In [ ]:
%%time
predictFromFile("./data/testing_users.csv","matrix_factorise_v9_k300_submission")

In [ ]:
predictFromFile("./data/200_users.csv","matrix_factorise_v9_k300_submission_200users")

In [ ]:
predictFromFile("./clean_data/test_before_021718_700row.csv","matrix_factorise_v8_k75_ontest700")

                               userCode ans_project_id
0  f5a8eabe-894a-1e9b-48bd-d70b063b3dd7   [5858, 5678]
1  b37b12ff-7227-f33f-3d20-41d6aa7bcb46         [9415]
2  a2157f10-69be-8a70-3731-0c57774d6b4e   [5386, 4437]
3  144ffbe5-a07d-b44d-6cf9-53e2f87f4354         [2610]
4  0b0317e4-1f20-c1c2-e81f-97a1a61ef841   [8437, 6519]


In [ ]:
for i in range(5):
    print(i)
    
    predictFromFile("./clean_data/test_data/test_before_021718_700row_fold%d.csv"%i,"matrix_factorise_v5_k5000_ontest700_fold%d"%i)

In [ ]:
%%time
users['project_id'] = users['userCode'].apply(recommend_project)

In [ ]:
users.to_csv("./submission/submission_v4_full.csv",index=False)

In [ ]:
users.head()

In [ ]:
users.shape